In [ ]:
import numpy as np 

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Diamond

In [ ]:
output_notebook()

In [ ]:
x, y = [1, 2, 3, 4, 5], [6, 7, 2, 4, 5]

In [ ]:
# My First Scatter Plot:

# create a new plot with default tools, using figure
p = figure()

# add a circle renderer with x and y coordinates
p.circle(x, y)

# radius=0.4   - diameter to graph scale
# size=45
# 'vectorize' sizes: 

show(p) # show the results

In [ ]:
p = figure(plot_height=200, sizing_mode="scale_width")

p.title.text = 'First Plot'
p.background_fill_color = 'beige'
p.outline_line_width = 7
p.outline_line_alpha = 0.3
p.outline_line_color = "navy"

r = p.square(x, y, color="firebrick")

r.glyph.size = 50
r.glyph.fill_alpha = 0.2
r.glyph.line_color = "firebrick"
r.glyph.line_dash = [5, 1]
r.glyph.line_width = 2


show(p) # show the results

## Modify Axis Styling

In [ ]:
from math import pi
p = figure(plot_height=200, sizing_mode="scale_width", tools='')
p.square(x, y, size=50)

# change just some things about the x-axes
p.xaxis.axis_label = "Temp"
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "red"
p.xaxis.major_label_orientation = pi / 3

# change just some things about the y-axes
p.yaxis.axis_label = "Pressure"
p.yaxis.major_label_text_color = "orange"
p.yaxis.major_label_orientation = "vertical"
p.yaxis.axis_line_color = "#00ff00"

# change things on all axes
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 6

show(p) # show the results

## Selecting Glyphs

In [ ]:
p = figure(plot_height=200, sizing_mode="scale_width", tools='tap', title='Select a Square')
renderer = p.circle(x, y, 
                        size=75,
#                         radius=0.5, # GOT-CHA - Hit testing doesn't quite work with radius on Circles

                    # set visual properties for selected glyphs
                    selection_color="firebrick",

                    # set visual properties for non-selected glyphs
                    nonselection_fill_alpha=0.2,
                    nonselection_fill_color="grey",
                    nonselection_line_color="firebrick",
                    nonselection_line_alpha=1.0)

show(p)
# Hold Shift / Ctrl for multi-select

## Glyph Types:
https://bokeh.pydata.org/en/latest/docs/reference/models/markers.html

<img src="images/bokeh_models_markers.png" width="200"/>


## Hovering Over Data

In [ ]:
from bokeh.plotting import ColumnDataSource


source = ColumnDataSource(data=dict(
    x=x,
    y=y,
    desc=['A', 'b', 'C', 'd', 'E'],
))

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y{0})"),
    ("desc", "@desc"),
]

p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

p.circle('x', 'y', size=40, source=source)

print(x)
print(y)
show(p)

In [ ]:
from bokeh.models.tools import HoverTool
from bokeh.sampledata.glucose import data

subset = data.loc['2010-10-06']

x, y = subset.index.to_series(), subset['glucose']

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", title='Hover over points')

p.line(x, y, line_dash="4 4", line_width=1, color='gray')

cr = p.circle(x, y, size=20,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white")

p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))

show(p)

In [ ]:
from bokeh.sampledata.stocks import AAPL
from bokeh.models import ColumnDataSource

tmp = AAPL
tmp['adj close'] = AAPL['adj_close']

tmp['date'] = np.array(AAPL['date'], dtype=np.datetime64) # convert date strings to real datetimes

p = figure(x_axis_type="datetime", title="AAPL", plot_height=250, sizing_mode='scale_width')
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'

p.line('date', 'adj close', source=ColumnDataSource(data=tmp), line_dash="dashed", line_color='grey')


ht = HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
        ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
        ( 'volume', '@volume{0.00 a}'      ),
    ],

    formatters={
        'date'      : 'datetime', # use 'datetime' formatter for 'date' field
        'adj close' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)
p.add_tools(ht)

show(p)
# len(tmp['date'])

## Annotations

In [ ]:
from bokeh.models.annotations import Arrow
from bokeh.models.arrow_heads import OpenHead, NormalHead, VeeHead # Arrow head Types
from bokeh.sampledata.stocks import AAPL
from bokeh.models import ColumnDataSource
from bokeh.models.annotations import BoxAnnotation


tmp = AAPL
tmp['adj close'] = AAPL['adj_close']

tmp['date'] = np.array(AAPL['date'], dtype=np.datetime64) # convert date strings to real datetimes

p = figure(x_axis_type="datetime", title="AAPL", plot_height=450, sizing_mode='scale_width')
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'

p.line('date', 'adj close', source=ColumnDataSource(data=tmp), line_dash="dashed", line_color='grey')

# x_start and _end need to be np.datetime64 data types to match the underlying data source
p.add_layout(Arrow(end=NormalHead(fill_color="red"),
                   x_start=np.datetime64('2000-11-14'), y_start=20, x_end=np.datetime64('2012-08-01'), y_end=680))

# region that always fills the top of the plot
upper = BoxAnnotation(bottom=600, fill_alpha=0.1, fill_color='olive')
p.add_layout(upper)

# region that always fills the bottom of the plot
lower = BoxAnnotation(top=200, fill_alpha=0.1, fill_color='black')
p.add_layout(lower)

# a finite region
center = BoxAnnotation(top=500, bottom=300, left=np.datetime64('2012-01-01'), right=np.datetime64('2014-01-01'), 
                       fill_alpha=0.1, fill_color='navy')
p.add_layout(center)

show(p)
# Can Also add lines, circles, elipsis, or other polygons

## Linking Graphs 

In [ ]:
from bokeh.layouts import gridplot

x = list(range(11))
y0, y1, y2 = x, [10-i for i in x], [abs(i-5) for i in x]

plot_options = dict(width=250, plot_height=250, tools='pan,wheel_zoom,box_select,reset')

# create a new plot
s1 = figure(**plot_options)
s1.circle(x, y0, size=10, color="navy")

# create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
s2.triangle(x, y1, size=10, color="firebrick")

# create a new plot and share only one range
s3 = figure(x_range=s1.x_range, **plot_options)
s3.square(x, y2, size=10, color="olive")

p = gridplot([[s1, s2, s3]])

# show the results
show(p)

In [ ]:
from bokeh.models import ColumnDataSource

x = list(range(-20, 21))
y0, y1 = [abs(xx) for xx in x], [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "wheel_zoom,box_select,lasso_select,reset"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

## Bar Charts

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]

source = ColumnDataSource(data=dict(fruits=fruits, counts=counts, color=Spectral6))

p = figure(x_range=fruits, plot_height=250, y_range=(0, 9), title="Fruit Counts")
p.vbar(x='fruits', top='counts', width=0.9, color='color', legend="fruits", source=source)

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

## Stacked Bar Chart

In [ ]:
from bokeh.palettes import GnBu3, OrRd3

years = ['2015', '2016', '2017']

exports = {'fruits' : fruits,
           '2015'   : [2, 1, 4, 3, 2, 4],
           '2016'   : [5, 3, 4, 2, 4, 6],
           '2017'   : [3, 2, 4, 4, 5, 3]}
imports = {'fruits' : fruits,
           '2015'   : [-1, 0, -1, -3, -2, -1],
           '2016'   : [-2, -1, -3, -1, -2, -2],
           '2017'   : [-1, -2, -1, 0, -2, -2]}

p = figure(y_range=fruits, plot_height=250, x_range=(-16, 16), title="Fruit import/export, by year")

p.hbar_stack(years, y='fruits', height=0.9, color=GnBu3, source=ColumnDataSource(exports),
             legend=["%s exports" % x for x in years])

p.hbar_stack(years, y='fruits', height=0.9, color=OrRd3, source=ColumnDataSource(imports),
             legend=["%s imports" % x for x in years])

p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.legend.location = "center_left"

show(p)

## Grouped Bar Chart

In [ ]:
from bokeh.models import FactorRange
from bokeh.transform import factor_cmap


fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year")


p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",

       # use the palette to colormap based on the the x[1:2] values
       fill_color=factor_cmap('x', palette=['firebrick', 'olive', 'navy'], factors=years, start=1, end=2))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

## Bar Chart Jitter

In [ ]:
from bokeh.sampledata.commits import data

# Github Commits
data.head()

In [ ]:
from bokeh.transform import jitter

DAYS = ['Sun', 'Sat', 'Fri', 'Thu', 'Wed', 'Tue', 'Mon']

source = ColumnDataSource(data)

p = figure(plot_width=800, plot_height=300, y_range=DAYS, x_axis_type='datetime', 
           title="Commits by Time of Day (US/Central) 2012—2016")

p.circle(x='time', y='day',  source=source, alpha=0.3)
# jitter('day', width=0.6, range=p.y_range)

p.xaxis[0].formatter.days = ['%Hh']
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None

show(p)

## Datashader

In [ ]:
import pandas as pd
import numpy as np
import datashader as ds
import datashader.transfer_functions as tf

np.random.seed(1)
num = 10000

dists = {
    cat: pd.DataFrame(dict(x=np.random.normal(x,s,num),
                                y=np.random.normal(y,s,num),
                                val=val,cat=cat))
         for x,y,s,val,cat in [(2,2,0.01,10,"d1"), (2,-2,0.1,20,"d2"), (-2,-2,0.5,30,"d3"), (-2,2,1.0,40,"d4"), (0,0,3,50,"d5")]
}

df = pd.concat(dists, ignore_index=True)
df["cat"] = df["cat"].astype("category")
df.tail()

In [ ]:
df.head()

In [ ]:
%time tf.shade(ds.Canvas().points(df,'x','y'))

In [ ]:
canvas = ds.Canvas(plot_width=250, plot_height=250, x_range=(-4,4), y_range=(-4,4))
agg = canvas.points(df, 'x', 'y', agg=ds.count())
agg

In [ ]:
agg = canvas.points(df, 'x', 'y', agg=ds.count())
%time tf.shade(ds.Canvas().points(df,'x','y', agg=ds.count_cat('cat')))

In [ ]:
import bokeh.plotting as bp
from datashader.bokeh_ext import InteractiveImage

bp.output_notebook()
p = bp.figure(tools='pan,wheel_zoom,reset,box_zoom', x_range=(-5,5), y_range=(-5,5))
color_key = dict(d1='blue', d2='yellow', d3='red', d4='orange', d5='olive')

def image_callback(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'x', 'y', ds.count_cat('cat'))
    img = tf.shade(agg, color_key)
    return tf.dynspread(img) #, threshold=0.25)

InteractiveImage(p, image_callback)